In [1]:
import logging
import sys

import numpy as np
import tvm
from tvm import te
import tvm.testing

from tvm import autotvm

标准矩阵乘法，通过TE表示

In [2]:
def matmul_basic(N, L, M, dtype):
    A = te.placeholder((N, L), name="A", dtype=dtype)
    B = te.placeholder((L, N), name="A", dtype=dtype)

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # schedule
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    # 这里将8作为tiling factor，但是8不一定是最优的参数
    # 在autotvm中，能否定义可以调整的参数（tunable parameter）
    yo, yi = s[C].split(y, 8)
    xo, xi = s[C].split(x, 8)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]


下面是一个简单的模版，其中包含一些可以调节的参数

In [3]:
# 将该函数标记为一个模版
@autotvm.template("tutorial/matmul_v1")
def matmul_v1(N, L, M, dtype):
    A = te.placeholder((N, L), name="A", dtype=dtype)
    B = te.placeholder((L, M), name="B", dtype=dtype)

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # schedule
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    # 2. get the config object
    # cfg表示config对象，而非控制流图
    cfg = autotvm.get_config()

    # 3. define search space
    # 两个搜索空间，名为tile_y和tile_x
    cfg.define_knob("tile_y", [1, 2, 4, 8, 16])
    cfg.define_knob("tile_x", [1, 2, 4, 8, 16])

    # 4. schedule according to config
    # 在构建的5*5的空间中进行调度
    yo, yi = s[C].split(y, cfg["tile_y"].val)
    xo, xi = s[C].split(x, cfg["tile_x"].val)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

下面是一种更高级、更智能的定义搜索空间的方法

In [4]:
@autotvm.template("tutorial/matmul")
def matmul(N, L, M, dtype):
    A = te.placeholder((N, L), name="A", dtype=dtype)
    B = te.placeholder((L, M), name="B", dtype=dtype)

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # schedule
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    # 开始定义空间
    cfg = autotvm.get_config()
    # 枚举出所有可能
    cfg.define_split("tile_y", y, num_outputs=2)
    cfg.define_split("tile_x", x, num_outputs=2)

    # 根据config进行调度
    yo, yi = cfg["tile_y"].apply(s, C, y)
    xo, xi = cfg["tile_x"].apply(s, C, x)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

使用AutoTVM优化矩阵乘法

根据上面编写的矩阵乘法模版，可对炒粉的schedule中的块的大小进行参数化，对上述模版定义的参数空间进行搜索，下面选择一个指导器对空间进行拆分

调优器可以使用如下策略进行搜索，根据空间大小的不同，可以使用不同的种类

如果空间非常大，可以选择最后一种，例如CUDA GPU上的conv2d蒜子搜索空间大小达到10^9级别

+ `tvm.autotvm.tuner.RandomTuner`：以随机顺序枚举空间
+ `tvm.autotvm.tuner.GridSearchTuner`：以网格搜索顺序枚举空间
+ `tvm.autotvm.tuner.GATuner`：使用遗传算法搜索空间
+ `tvm.autotvm.tuner.XGBTuner`：用基于模型的方法训练一个 XGBoost 模型，来预测降级 IR 的速度，并根据预测值选择下一批配置

下面是调优过程，首先创建一个调优任务，然后检查初始的搜索空间

In [5]:
N, L, M = 512, 512, 512
task = autotvm.task.create("tutorial/matmul", args=(N, L, M, "float32"), target="llvm")
print(task.config_space)

ConfigSpace (len=100, range_length=100, space_map=
   0 tile_y: Split(policy=factors, product=512, num_outputs=2) len=10
   1 tile_x: Split(policy=factors, product=512, num_outputs=2) len=10
)


In [6]:
logging.getLogger("autotvm").setLevel(logging.DEBUG)
logging.getLogger("autotvm").addHandler(logging.StreamHandler(sys.stdout))

下面来进行评估，评估有两个步骤，构建和运行，默认用所有CPU core来编译，然后依次进行评估，为了减少方差，对5次平局结果取平均值

In [7]:
measure_option = autotvm.measure_option(builder="local", runner=autotvm.LocalRunner(number=5))

# 用RandomTuner开始调优，日志记录到`matmul.log`文件中
# 可用XGBTuner来替代
tuner = autotvm.tuner.RandomTuner(task)
tuner.tune(
    n_trial=10,
    measure_option=measure_option,
    # 通过回调函数写入log
    callbacks=[autotvm.callback.log_to_file("matmul.log")],
)


waiting for device...
device available
Get devices for measurement successfully!
No: 1	GFLOPS: 8.10/8.10	result: MeasureResult(costs=(0.0331215282,), error_no=MeasureErrorNo.NO_ERROR, all_cost=0.7650084495544434, timestamp=1677583178.9302762)	[('tile_y', [-1, 64]), ('tile_x', [-1, 8])],None,36
No: 2	GFLOPS: 15.03/15.03	result: MeasureResult(costs=(0.017860347,), error_no=MeasureErrorNo.NO_ERROR, all_cost=0.4784693717956543, timestamp=1677583179.4750073)	[('tile_y', [-1, 512]), ('tile_x', [-1, 32])],None,59
No: 3	GFLOPS: 28.17/28.17	result: MeasureResult(costs=(0.009529478000000001,), error_no=MeasureErrorNo.NO_ERROR, all_cost=0.3636972904205322, timestamp=1677583180.1045558)	[('tile_y', [-1, 2]), ('tile_x', [-1, 64])],None,61
No: 4	GFLOPS: 3.23/28.17	result: MeasureResult(costs=(0.0831133974,), error_no=MeasureErrorNo.NO_ERROR, all_cost=1.528329610824585, timestamp=1677583181.9062355)	[('tile_y', [-1, 64]), ('tile_x', [-1, 4])],None,26
No: 5	GFLOPS: 9.35/28.17	result: MeasureResult(cos

从日志文件中选择具有最佳性能的配置，并用相应的参数来编译schedule

可直接在`autotvm.apply_history_best`上下文中调用`matmul`函数，它会用参数查询调度上下文，然后可用相同的参数获取最优配置

In [8]:
# 从日志文件中应用历史最佳
with autotvm.apply_history_best("matmul.log"):
    with tvm.target.Target("llvm"):
        s, arg_bufs = matmul(N, L, M, "float32")
        func = tvm.build(s, arg_bufs)

# 验证正确性
a_np = np.random.uniform(size=(N, L)).astype(np.float32)
b_np = np.random.uniform(size=(L, M)).astype(np.float32)
c_np = a_np.dot(b_np)

c_tvm = tvm.nd.empty(c_np.shape)
func(tvm.nd.array(a_np), tvm.nd.array(b_np), c_tvm)

tvm.testing.assert_allclose(c_np, c_tvm.numpy(), rtol=1e-4)

Finish loading 10 records
